In [12]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

import keras
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Activation, Dropout, Input, concatenate, Reshape
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, MaxPooling2D, UpSampling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.layers.embeddings import Embedding
from keras import losses, optimizers

from keras.datasets import mnist

import keras.backend as K

In [14]:
class BIGAN:
    def __init__(self):
        img_rows = 28
        img_cols = 28
        img_channels = 1
        self.img_shape = (img_rows, img_cols, img_channels)
        self.latent_dim = 100
        
        # optimizer
        optimizer = optimizers.Adam(lr = 0.0002, beta_1 = 0.5, beta_2 = 0.999)
        
        # build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss = losses.binary_crossentropy, optimizer = optimizer, metrics = ['accuracy'])
        
        self.generator = self.build_generator()
        self.encoder = self.build_encoder()
        
        # The part of the bigan that trains the discriminator and encoder
        self.discriminator.trainable = False
        
        # Generate image from sampled noise
        z = Input(shape = (self.latent_dim, ))
        img_ = self.generator(z)
        
        # encode image
        img = Input(shape = self.img_shape)
        z_ = self.encoder(img)
        
        # Latent -> img is fake, and img -> latent is valid
        fake = self.discriminator([z, img_])
        valid = self.discriminator([z_, img])
        
        self.bigan_generator = Model(inputs = [z, img], outputs = [fake, valid])
        self.bigan_generator.compile(loss = [losses.binary_crossentropy, losses.binary_crossentropy], optimizer = optimizer)
        
    def build_encoder(self):
        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(self.latent_dim))

        model.summary()

        img = Input(shape=self.img_shape)
        z = model(img)

        return Model(img, z)

    def build_generator(self):
        model = Sequential()

        model.add(Dense(512, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        z = Input(shape=(self.latent_dim,))
        gen_img = model(z)

        return Model(z, gen_img)

    def build_discriminator(self):

        z = Input(shape=(self.latent_dim, ))
        img = Input(shape=self.img_shape)
        d_in = concatenate([z, Flatten()(img)])

        model = Dense(1024)(d_in)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.5)(model)
        model = Dense(1024)(model)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.5)(model)
        model = Dense(1024)(model)
        model = LeakyReLU(alpha=0.2)(model)
        model = Dropout(0.5)(model)
        validity = Dense(1, activation="sigmoid")(model)

        return Model([z, img], validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        
        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):


            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Sample noise and generate img
            z = np.random.normal(size=(batch_size, self.latent_dim))
            imgs_ = self.generator.predict(z)

            # Select a random batch of images and encode
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]
            z_ = self.encoder.predict(imgs)
            
            # Train the discriminator (img -> z is valid, z -> img is fake)
            d_loss_real = self.discriminator.train_on_batch([z_, imgs], valid)
            d_loss_fake = self.discriminator.train_on_batch([z, imgs_], fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (z -> img is valid and img -> z is is invalid)
            g_loss = self.bigan_generator.train_on_batch([z, imgs], [valid, fake])

            # Plot the progress
            print ("%d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0]))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_interval(epoch)

    def sample_interval(self, epoch):
        r, c = 5, 5
        z = np.random.normal(size=(25, self.latent_dim))
        gen_imgs = self.generator.predict(z)

        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/mnist_%d.png" % epoch)
        plt.close()

In [15]:
bigan = BIGAN()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 512)               51712     
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 512)               2048      
_________________________________________________________________
dense_24 (Dense)             (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 512)               2048      
_________________________________________________________________
dense_25 (Dense)             (None, 784)               402192    
__________

In [ ]:
bigan.train(epochs=40000, batch_size=32, sample_interval=400)

/home/akash/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.209561, acc: 29.69%] [G loss: 3.521557]
1 [D loss: 0.537124, acc: 68.75%] [G loss: 4.312933]
2 [D loss: 0.297156, acc: 85.94%] [G loss: 5.388489]
3 [D loss: 0.145218, acc: 98.44%] [G loss: 5.820424]
4 [D loss: 0.124021, acc: 98.44%] [G loss: 6.782402]
5 [D loss: 0.069717, acc: 100.00%] [G loss: 7.288142]
6 [D loss: 0.052402, acc: 100.00%] [G loss: 8.190542]
7 [D loss: 0.044468, acc: 100.00%] [G loss: 7.759377]
8 [D loss: 0.046543, acc: 100.00%] [G loss: 8.893176]
9 [D loss: 0.028342, acc: 100.00%] [G loss: 10.106620]
10 [D loss: 0.028409, acc: 100.00%] [G loss: 10.295261]
11 [D loss: 0.021820, acc: 100.00%] [G loss: 10.885851]
12 [D loss: 0.024410, acc: 100.00%] [G loss: 11.563570]
13 [D loss: 0.010985, acc: 100.00%] [G loss: 11.019537]
14 [D loss: 0.010540, acc: 100.00%] [G loss: 11.423648]
15 [D loss: 0.014244, acc: 100.00%] [G loss: 11.831979]
16 [D loss: 0.010784, acc: 100.00%] [G loss: 12.134274]
17 [D loss: 0.008301, acc: 100.00%] [G loss: 11.608919]
18 [D loss: 0.01